In [1]:
# Celda 1: Setup
import mlflow
from pathlib import Path
import sys
import json

sys.path.insert(0, str(Path.cwd().parent))

from src.property_analysis.text_analyzer import PropertyTextAnalyzer

# MLflow
project_root = Path("/Users/santiagoamaya/Desktop/propAgent/real-estate-ai-finder")
mlflow.set_tracking_uri(f"file://{project_root}/mlruns")
mlflow.set_experiment("text_analysis_experiments")

print("✅ Setup complete")

/opt/anaconda3/envs/rai/lib/python3.10/site-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/12/02 19:45:48 INFO mlflow.tracking.fluent: Experiment with name 'text_analysis_experiments' does not exist. Creating a new experiment.


✅ Setup complete


In [2]:
# Celda 2: Cargar propiedades reales
import json

# Cargar tus datos reales
with open('../data/raw/fotocasa_20251123_154917.json') as f:
    data = json.load(f)

properties = data['properties']
print(f"📊 Loaded {len(properties)} properties")

# Mostrar ejemplo
sample = properties[0]
print(f"\n📝 Sample property:")
print(f"   ID: {sample['id']}")
print(f"   Description: {sample['description'][:200]}...")

📊 Loaded 6 properties

📝 Sample property:
   ID: 187843062
   Description: Encantador piso señorial reformado con gusto y detalles originales en el corazón de Barcelona

En una finca señorial de 1892, se encuentra este magnífico piso de 103 m², que combina el encanto de la a...


In [7]:
# Celda 3: Analizar propiedades
analyzer = PropertyTextAnalyzer(prompt_version="v2.0")

with mlflow.start_run(run_name="baseline_v1.0_real_data"):
    results = []
    
    for prop in properties[:5]:  # Primeras 5 para testing
        result = analyzer.analyze(
            property_id=prop['id'],
            description=prop.get('description', '')
        )
        results.append(result)
        
        print(f"\n{'='*60}")
        print(f"🏠 Property: {prop['id']}")
        print(f"📍 Location: {prop['location']}")
        print(f"💰 Price: {prop['price']:,}€")
        print(f"\n📝 Description (first 200 chars):")
        print(f"   {prop.get('description', 'N/A')[:200]}...")
        
        detected = result.detected_features  # Es una lista directamente
        detected = [f for f in detected if f.confidence >= 0.5]
        if detected:
            print(f"\n✅ Detected features (score >= 0.5):")
            for feature, score in sorted(detected, key=lambda x: x[1], reverse=True):
                print(f"   {feature}: {score:.2f}")
        else:
            print(f"\n⚠️  No features detected with confidence >= 0.5")
        
        print(f"\n📊 Overall confidence: {result.features.confidence:.2f}")
    
    # Log aggregate metrics
    avg_confidence = sum(r.features.confidence for r in results) / len(results)
    mlflow.log_metric("avg_confidence", avg_confidence)
    mlflow.log_param("prompt_version", "v1.0")
    mlflow.log_param("properties_analyzed", len(results))
    
print(f"\n✅ Average confidence: {avg_confidence:.2f}")

✅ Analyzer ready (backend: api)

🏠 Property: 187843062
📍 Location: Sant Antoni, Eixample, Barcelona Capital
💰 Price: 449,000€

📝 Description (first 200 chars):
   Encantador piso señorial reformado con gusto y detalles originales en el corazón de Barcelona

En una finca señorial de 1892, se encuentra este magnífico piso de 103 m², que combina el encanto de la a...

✅ Detected features (score >= 0.5):


TypeError: 'DetectedFeature' object is not subscriptable

In [ ]:
# Celda 4: Análisis de cobertura
import pandas as pd

# Crear DataFrame con resultados
data_analysis = []
for result in results:
    detected = result.detected_features  # Es una lista directamente
    detected = [f for f in detected if f.confidence >= 0.5]
    data_analysis.append({
        'property_id': result.property_id,
        'num_features': len(detected),
        'confidence': result.features.confidence,
        'features': ', '.join(detected.keys()) if detected else 'none'
    })

df_results = pd.DataFrame(data_analysis)
print(df_results)

print(f"\n📊 Coverage Statistics:")
print(f"   Avg features per property: {df_results['num_features'].mean():.1f}")
print(f"   Properties with 0 features: {(df_results['num_features'] == 0).sum()}")
print(f"   Properties with 3+ features: {(df_results['num_features'] >= 3).sum()}")

AttributeError: 'PropertyAnalysis' object has no attribute 'features'